<a href="https://colab.research.google.com/github/guifav/roger/blob/main/Roger%20-%20AI_Agente_Escritor_de_Blog%20(Com%20SERPER).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import re
import json
import time
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
from datetime import datetime
from IPython.display import HTML, display, Markdown

# Verificar se estamos no ambiente do Google Colab
try:
    from google.colab import userdata
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Instalação das bibliotecas necessárias
try:
    from openai import OpenAI
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
except ImportError:
    !pip install openai==1.3.0 nltk tqdm pandas requests
    from openai import OpenAI
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords

# Download dos pacotes NLTK necessários
nltk.download('punkt')
nltk.download('stopwords')

# Garantir que o modelo para o português está disponível
try:
    nltk.data.find('tokenizers/punkt/portuguese.pickle')
except LookupError:
    # Tenta baixar o pacote específico para português
    nltk.download('punkt', download_dir='/root/nltk_data')

    # Caso ainda ocorra erro, usaremos o tokenizador básico
    print("Aviso: Usando tokenizador básico para português.")

# Classe SerperSearchAPI implementada anteriormente
class SerperSearchAPI:
    """Classe para realizar buscas na internet usando a API Serper."""

    def __init__(self, api_key=None):
        """
        Inicializa a API Serper.

        Args:
            api_key (str, optional): Chave da API Serper.
        """
        self.api_key = api_key
        self.base_url = "https://google.serper.dev/search"
        self.headers = None

    def configure_api(self):
        """Configura a API do Serper."""
        if not self.api_key:
            # Verifica se estamos no Google Colab e tenta obter a chave do userdata
            if IN_COLAB:
                try:
                    self.api_key = userdata.get('SERPER_API_KEY')
                    print("✅ Chave API Serper obtida com sucesso do armazenamento seguro do Colab.")
                except Exception as e:
                    print(f"⚠️ Não foi possível obter a chave Serper API do armazenamento seguro: {str(e)}")
                    print("Para configurar uma chave secreta no Google Colab:")
                    print("1. No menu lateral, clique em '🔑' (Secrets)")
                    print("2. Adicione uma nova chave secreta com nome 'SERPER_API_KEY' e seu valor")
                    self.api_key = input("Digite sua chave de API Serper manualmente: ")
            else:
                self.api_key = input("Digite sua chave de API Serper: ")

        self.headers = {
            "X-API-KEY": self.api_key,
            "Content-Type": "application/json"
        }

    def search(self, query, num_results=5):
        """
        Realiza uma busca com a query fornecida.

        Args:
            query (str): Query de busca
            num_results (int): Número de resultados a retornar

        Returns:
            dict: Resultados da busca ou erro
        """
        if not self.headers:
            self.configure_api()

        payload = {
            "q": query,
            "num": num_results
        }

        try:
            response = requests.post(
                self.base_url,
                headers=self.headers,
                json=payload
            )

            if response.status_code == 200:
                return response.json()
            else:
                return {
                    "error": f"Erro na busca: {response.status_code}",
                    "details": response.text
                }

        except Exception as e:
            return {"error": f"Erro ao realizar a busca: {str(e)}"}

    def extract_search_results(self, results):
      """
      Extrai informações relevantes dos resultados de busca.

      Args:
          results (dict): Resultados da busca

      Returns:
          list: Lista com informações relevantes extraídas
      """
      if isinstance(results, dict) and "error" in results:
          return [{"title": "Erro na busca", "snippet": results["error"], "type": "error"}]

      extracted = []

      # Resultados orgânicos
      if isinstance(results, dict) and "organic" in results:
          for item in results["organic"]:
              extracted.append({
                  "title": item.get("title", ""),
                  "link": item.get("link", ""),
                  "snippet": item.get("snippet", ""),
                  "type": "organic"
              })

      # Knowledge graph
      if isinstance(results, dict) and "knowledgeGraph" in results:
          kg = results["knowledgeGraph"]
          extracted.append({
              "title": kg.get("title", ""),
              "type": kg.get("type", "Informação"),
              "description": kg.get("description", ""),
              "type": "knowledge"
          })

      # Resultados de notícias
      if isinstance(results, dict) and "news" in results:
          for item in results["news"]:
              extracted.append({
                  "title": item.get("title", ""),
                  "link": item.get("link", ""),
                  "snippet": item.get("snippet", ""),
                  "date": item.get("date", ""),
                  "type": "news"
              })

      return extracted

    def format_search_results_for_openai(self, results, max_results=5):
      """
      Formata os resultados da busca para uso com a OpenAI.

      Args:
          results (list): Lista de resultados extraídos
          max_results (int): Número máximo de resultados a formatar

      Returns:
          str: Texto formatado com os resultados
      """
      if not results:
          return "Nenhum resultado encontrado."

      if isinstance(results, list) and len(results) > 0 and "type" in results[0] and results[0]["type"] == "error":
          return f"Erro na busca: {results[0]['snippet']}"

      formatted = "### Resultados da pesquisa\n\n"

      # Limitando o número de resultados
      results = results[:max_results]

      for i, item in enumerate(results, 1):
          if item["type"] == "organic" or item["type"] == "news":
              formatted += f"{i}. **{item['title']}**\n"
              formatted += f"   - Link: {item.get('link', 'N/A')}\n"
              formatted += f"   - Snippet: {item.get('snippet', 'N/A')}\n"
              if "date" in item and item["date"]:
                  formatted += f"   - Data: {item['date']}\n"
              formatted += "\n"
          elif item["type"] == "knowledge":
              formatted += f"{i}. **{item['title']}** ({item.get('type', 'N/A')})\n"
              formatted += f"   - Descrição: {item.get('description', 'N/A')}\n\n"

      formatted += f"Resultados obtidos em {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n"

      return formatted

class BlogArticleGenerator:
    def __init__(self, openai_api_key=None, serper_api_key=None, model="gpt-3.5-turbo"):
        """
        Inicializa o gerador de artigos para blog.

        Args:
            openai_api_key (str, optional): Chave da API OpenAI.
            serper_api_key (str, optional): Chave da API Serper para busca na internet.
            model (str): Modelo da OpenAI a ser usado.
        """
        self.openai_api_key = openai_api_key
        self.serper_api_key = serper_api_key
        self.min_word_count = 2000
        self.model = model
        self.client = None
        self.search_api = None
        self.use_internet_search = False
        self.article = None
        self.metadata = {
            "title": "",
            "keywords": [],
            "word_count": 0,
            "seo_score": 0,
            "readability_score": 0
        }
        self.configure_api()

    def configure_api(self):
        """Configura a API do OpenAI."""
        if not self.openai_api_key:
            # Verifica se estamos no Google Colab e tenta obter a chave do userdata
            if IN_COLAB:
                try:
                    self.openai_api_key = userdata.get('OPENAI_API_KEY')
                    print("✅ Chave API OpenAI obtida com sucesso do armazenamento seguro do Colab.")
                except Exception as e:
                    print(f"⚠️ Não foi possível obter a chave API do armazenamento seguro: {str(e)}")
                    print("Para configurar uma chave secreta no Google Colab:")
                    print("1. No menu lateral, clique em '🔑' (Secrets)")
                    print("2. Adicione uma nova chave secreta com nome 'OPENAI_API_KEY' e seu valor")
                    self.openai_api_key = input("Digite sua chave de API OpenAI manualmente: ")
            else:
                self.openai_api_key = input("Digite sua chave de API OpenAI: ")

        self.client = OpenAI(api_key=self.openai_api_key)

        # Configuração da API Serper se disponível
        if self.serper_api_key or self.use_internet_search:
            self.search_api = SerperSearchAPI(api_key=self.serper_api_key)
            self.use_internet_search = True

    def enable_internet_search(self, serper_api_key=None):
        """
        Habilita a busca na internet usando a API Serper.

        Args:
            serper_api_key (str, optional): Chave da API Serper.
        """
        if serper_api_key:
            self.serper_api_key = serper_api_key

        self.search_api = SerperSearchAPI(api_key=self.serper_api_key)
        self.use_internet_search = True
        print("✅ Busca na internet habilitada usando API Serper.")

    def search_for_topic(self, topic, keywords, num_results=5):
        """
        Realiza uma busca na internet sobre o tópico e palavras-chave.

        Args:
            topic (str): Tópico principal do artigo.
            keywords (list): Lista de palavras-chave.
            num_results (int): Número de resultados a retornar.

        Returns:
            str: Texto formatado com os resultados da busca.
        """
        if not self.use_internet_search or not self.search_api:
            return "Busca na internet não habilitada."

        # Cria uma query de busca combinando o tópico e palavras-chave
        search_query = f"{topic} {' '.join(keywords)}"

        # Realiza a busca
        print(f"🔍 Buscando informações sobre: '{search_query}'")
        results = self.search_api.search(search_query, num_results)

        # Extrai e formata os resultados
        extracted_results = self.search_api.extract_search_results(results)
        formatted_results = self.search_api.format_search_results_for_openai(extracted_results)

        return formatted_results

    def create_outline(self, topic, keywords, tone="informative", use_search=True):
        """
        Cria um esboço detalhado para o artigo.

        Args:
            topic (str): Tópico principal do artigo.
            keywords (list): Lista de palavras-chave para SEO.
            tone (str): Tom do artigo (informativo, persuasivo, etc).
            use_search (bool): Se deve usar busca na internet para enriquecer o conteúdo.

        Returns:
            dict: Estrutura do artigo com título e seções.
        """
        # Busca informações na internet se habilitado
        search_results = ""
        if use_search and self.use_internet_search:
            search_results = self.search_for_topic(topic, keywords)

        prompt = f"""
        Você é um especialista em criação de conteúdo para blogs otimizados para SEO.

        Crie um esboço detalhado para um artigo de blog sobre "{topic}" com tom {tone}.

        O artigo deve ter pelo menos 2000 palavras e incorporar as seguintes palavras-chave de forma natural:
        {', '.join(keywords)}

        O esboço deve incluir:
        1. Um título atraente e otimizado para SEO (menos de 70 caracteres)
        2. Uma meta-descrição atraente (150-160 caracteres)
        3. Uma introdução convincente
        4. Pelo menos 5-7 seções principais com subtítulos descritivos e otimizados para SEO
        5. Para cada seção principal, liste 3-5 pontos-chave a serem abordados
        6. Uma conclusão forte
        7. Uma chamada para ação eficaz

        {search_results if search_results else ""}

        Formate sua resposta como um JSON com a seguinte estrutura:
        {{
            "title": "Título do Artigo",
            "meta_description": "Meta descrição atrativa para SEO",
            "sections": [
                {{
                    "heading": "Introdução",
                    "points": ["ponto 1", "ponto 2", "ponto 3"]
                }},
                {{
                    "heading": "Seção 1",
                    "points": ["ponto 1", "ponto 2", "ponto 3"]
                }},
                ...
            ]
        }}
        """

        try:
            # Adiciona uma pausa de 1 segundo para evitar rate limiting
            time.sleep(1)

            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            outline_text = response.choices[0].message.content
            # Extrair apenas o JSON da resposta
            outline_json = re.search(r'({.*})', outline_text, re.DOTALL)
            if outline_json:
                try:
                    outline = json.loads(outline_json.group(1))
                except json.JSONDecodeError:
                    # Tenta limpar o texto para torná-lo um JSON válido
                    cleaned_json = outline_json.group(1).replace("'", '"').replace("\\", "\\\\")
                    try:
                        outline = json.loads(cleaned_json)
                    except:
                        print(f"Erro ao processar a resposta como JSON. Resposta recebida:\n{outline_text}")
                        return None
            else:
                try:
                    outline = json.loads(outline_text)
                except json.JSONDecodeError:
                    # Se não for possível extrair o JSON, tenta criar um esboço manualmente
                    print(f"Não foi possível extrair o JSON da resposta. Criando esboço manualmente.")
                    # Procura por um título
                    title_match = re.search(r'[tT]ítulo:?\s*(.+?)[\n\r]', outline_text)
                    title = title_match.group(1) if title_match else "Artigo sobre " + topic

                    # Cria seções a partir de marcadores ou números
                    sections_text = re.split(r'[\n\r]+\d+\.|\*\*|##', outline_text)
                    sections = []

                    for section in sections_text:
                        if len(section.strip()) > 10:  # Ignora seções muito curtas
                            parts = section.split('\n', 1)
                            heading = parts[0].strip().replace('*', '').replace('#', '')
                            content = parts[1] if len(parts) > 1 else ""

                            # Extrai pontos de marcadores
                            points = re.findall(r'[\n\r]+[•\-\*]\s*(.+?)[\n\r]', content)
                            if not points:
                                # Se não encontrar marcadores, divide por frases
                                sentences = re.findall(r'([^.!?]+[.!?])', content)
                                points = sentences[:3] if sentences else ["Detalhes sobre " + heading]

                            sections.append({
                                "heading": heading,
                                "points": points
                            })

                    outline = {
                        "title": title,
                        "meta_description": "Artigo informativo sobre " + topic,
                        "sections": sections
                    }

            self.metadata["title"] = outline["title"]
            self.metadata["keywords"] = keywords

            return outline

        except Exception as e:
            print(f"Erro ao criar o esboço: {e}")
            return None

    def generate_section(self, heading, points, keywords, previous_content="", use_search=True):
        """
        Gera o conteúdo de uma seção específica do artigo.

        Args:
            heading (str): Título da seção.
            points (list): Pontos chave a serem abordados na seção.
            keywords (list): Palavras-chave para SEO.
            previous_content (str): Conteúdo já gerado, para manter consistência.
            use_search (bool): Se deve usar busca na internet para enriquecer o conteúdo.

        Returns:
            str: Conteúdo da seção.
        """
        # Busca informações na internet se habilitado
        search_results = ""
        if use_search and self.use_internet_search:
            # Cria uma query de busca específica para a seção
            search_query = f"{heading} {' '.join(points[:2])} {' '.join(keywords[:2])}"
            results = self.search_api.search(search_query, 3)
            extracted_results = self.search_api.extract_search_results(results)
            search_results = self.search_api.format_search_results_for_openai(extracted_results)

        prompt = f"""
        Você é um redator especializado em conteúdo para blogs otimizados para SEO.

        Escreva uma seção detalhada para um artigo de blog com o título "{heading}".

        Aborde os seguintes pontos de forma abrangente e envolvente:
        {', '.join(points)}

        Incorpore naturalmente as seguintes palavras-chave quando for apropriado:
        {', '.join(keywords)}

        {search_results if search_results else ""}

        Algumas orientações:
        - Escreva em português brasileiro formal, mas conversacional
        - Use parágrafos curtos (3-4 frases)
        - Use subtítulos H3 quando necessário para dividir seções longas
        - Inclua exemplos concretos e específicos
        - Use linguagem ativa e envolvente
        - Evite jargões desnecessários
        - Certifique-se de que o conteúdo flua naturalmente e seja fácil de ler
        - No mínimo 300-400 palavras para esta seção

        {f"Mantenha consistência com o conteúdo anterior: {previous_content[:300]}..." if previous_content else ""}
        """

        try:
            # Adiciona uma pausa de 1 segundo para evitar rate limiting
            time.sleep(1)

            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            section_content = response.choices[0].message.content
            return section_content

        except Exception as e:
            print(f"Erro ao gerar a seção '{heading}': {e}")
            return ""

    def generate_full_article(self, outline, use_search=True):
        """
        Gera o artigo completo baseado no esboço.

        Args:
            outline (dict): Esboço estruturado do artigo.
            use_search (bool): Se deve usar busca na internet para enriquecer o conteúdo.

        Returns:
            str: Artigo completo formatado.
        """
        full_article = f"# {outline['title']}\n\n"
        previous_content = ""

        print("Gerando artigo completo...")
        for section in tqdm(outline['sections']):
            heading = section['heading']
            points = section['points']

            # Determina o nível de cabeçalho (Introdução e Conclusão são H2, o resto H2 também)
            header_level = "##"

            section_content = self.generate_section(
                heading,
                points,
                self.metadata["keywords"],
                previous_content,
                use_search
            )

            full_article += f"{header_level} {heading}\n\n{section_content}\n\n"
            previous_content += section_content

        self.article = full_article
        self.count_words()

        # Verifica se o tamanho mínimo foi atingido
        if self.metadata["word_count"] < self.min_word_count:
            print(f"O artigo tem apenas {self.metadata['word_count']} palavras. Expandindo para atingir o mínimo de {self.min_word_count}...")
            self.expand_article(use_search)

        return self.article

    def expand_article(self, use_search=True):
        """
        Expande o artigo para atingir o número mínimo de palavras.

        Args:
            use_search (bool): Se deve usar busca na internet para enriquecer o conteúdo.
        """
        words_needed = self.min_word_count - self.metadata["word_count"]
        sections = self.article.split("##")

        # Identifica as seções mais curtas para expandir
        section_lengths = [(i, len(section.split())) for i, section in enumerate(sections[1:])]
        section_lengths.sort(key=lambda x: x[1])

        # Expande as seções mais curtas
        for idx, _ in section_lengths[:3]:
            if words_needed <= 0:
                break

            # Extrai título e conteúdo da seção
            section = sections[idx + 1]
            section_parts = section.split("\n\n", 1)
            if len(section_parts) < 2:
                continue

            heading = section_parts[0].strip()
            content = section_parts[1]

            # Busca informações adicionais na internet se habilitado
            search_results = ""
            if use_search and self.use_internet_search:
                search_query = f"{heading} detalhado {' '.join(self.metadata['keywords'][:2])}"
                results = self.search_api.search(search_query, 3)
                extracted_results = self.search_api.extract_search_results(results)
                search_results = self.search_api.format_search_results_for_openai(extracted_results)

            # Gera conteúdo adicional
            prompt = f"""
            Expanda o seguinte conteúdo de blog para adicionar mais detalhes, exemplos e informações úteis.
            Adicione pelo menos 300 palavras de conteúdo relevante, mantendo o estilo e tom do texto original.
            Incorpore naturalmente as palavras-chave: {', '.join(self.metadata["keywords"])}

            {search_results if search_results else ""}

            Seção: {heading}

            Conteúdo atual:
            {content}
            """

            try:
                # Adiciona uma pausa de 1 segundo para evitar rate limiting
                time.sleep(1)

                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7
                )

                expanded_content = response.choices[0].message.content
                sections[idx + 1] = f"{heading}\n\n{expanded_content}"
                words_needed -= 300  # Estimativa de palavras adicionadas

            except Exception as e:
                print(f"Erro ao expandir a seção: {e}")

        # Reúne o artigo expandido
        self.article = sections[0] + "##".join(sections[1:])
        self.count_words()

    def count_words(self):
        """Conta o número de palavras no artigo."""
        if self.article:
            # Usa um método mais simples para contar palavras, evitando erros de tokenização
            try:
                words = word_tokenize(self.article, language='portuguese')
            except LookupError:
                # Método alternativo de contagem de palavras
                words = self.article.split()

            # Remove pontuação e outros caracteres não-palavras
            words = [word for word in words if re.match(r'\w+', word)]
            self.metadata["word_count"] = len(words)

    def calculate_seo_score(self):
        """
        Calcula uma pontuação de SEO para o artigo.

        Returns:
            float: Pontuação de SEO entre 0 e 100.
        """
        if not self.article:
            return 0

        score = 0
        text_lower = self.article.lower()

        # Verifica o uso de palavras-chave
        for keyword in self.metadata["keywords"]:
            keyword_lower = keyword.lower()
            count = text_lower.count(keyword_lower)

            # Densidade de palavras-chave ideal (0.5% - 2%)
            density = count / self.metadata["word_count"] * 100
            if 0.5 <= density <= 2.0:
                score += 20 / len(self.metadata["keywords"])
            elif density > 0:
                score += 10 / len(self.metadata["keywords"])

        # Verifica presença de palavras-chave em títulos
        title_score = 0
        title_lines = re.findall(r'#{1,3}\s+(.+)', self.article)
        for title in title_lines:
            title_lower = title.lower()
            for keyword in self.metadata["keywords"]:
                if keyword.lower() in title_lower:
                    title_score += 1

        # Normaliza a pontuação de títulos (max 20 pontos)
        title_score = min(title_score, 20)
        score += title_score

        # Verifica tamanho do artigo
        if self.metadata["word_count"] >= self.min_word_count:
            score += 20
        elif self.metadata["word_count"] >= self.min_word_count * 0.8:
            score += 10

        # Verifica estrutura do artigo (presença de H2, H3)
        if len(re.findall(r'##\s+', self.article)) >= 4:
            score += 20
        else:
            h2_count = len(re.findall(r'##\s+', self.article))
            score += h2_count * 5

        # Verifica presença de links (não implementado aqui, precisaria de mais contexto)
        # Para uma implementação completa, você poderia adicionar verificação de links

        # Verifica comprimento dos parágrafos (ideal: 3-4 frases)
        paragraphs = re.findall(r'\n\n(.+?)\n\n', self.article, re.DOTALL)
        short_paragraphs = 0
        for para in paragraphs:
            sentences = re.findall(r'[.!?]+', para)
            if 1 <= len(sentences) <= 4:
                short_paragraphs += 1

        if len(paragraphs) > 0:
            paragraph_score = short_paragraphs / len(paragraphs) * 20
            score += paragraph_score

        self.metadata["seo_score"] = round(score, 2)
        return self.metadata["seo_score"]

    def calculate_readability_score(self):
        """
        Calcula uma pontuação de legibilidade para o artigo.

        Returns:
            float: Pontuação de legibilidade entre 0 e 100.
        """
        if not self.article:
            return 0

        # Tokeniza o texto de maneira segura
        try:
            tokens = word_tokenize(self.article, language='portuguese')
        except LookupError:
            # Método alternativo de tokenização
            tokens = [word for word in self.article.split() if re.match(r'\w+', word)]

        if not tokens:
            return 0

        # Calcula o comprimento médio das palavras
        avg_word_length = sum(len(word) for word in tokens) / len(tokens)

        # Estima o número de frases usando regex
        sentences = re.findall(r'[^.!?]+[.!?]', self.article)
        num_sentences = len(sentences) if sentences else 1

        # Calcula palavras por frase (média)
        words_per_sentence = self.metadata["word_count"] / max(num_sentences, 1)

        # Pontuação baseada em simplicidade de palavras (quanto menor o comprimento médio, melhor, até certo ponto)
        word_length_score = 50 - (abs(avg_word_length - 5) * 10)
        word_length_score = max(0, min(50, word_length_score))

        # Pontuação baseada no comprimento das frases (ideal: 15-20 palavras por frase)
        sentence_length_score = 50 - (abs(words_per_sentence - 17.5) * 2)
        sentence_length_score = max(0, min(50, sentence_length_score))

        readability_score = word_length_score + sentence_length_score
        self.metadata["readability_score"] = round(readability_score, 2)

        # Para referências nas sugestões de melhoria
        self.avg_word_length = avg_word_length
        self.words_per_sentence = words_per_sentence

        return self.metadata["readability_score"]

    def analyze_article(self):
        """
        Analisa o artigo e retorna métricas.

        Returns:
            dict: Metadados e pontuações do artigo.
        """
        if not self.article:
            return {"error": "Nenhum artigo gerado ainda."}

        self.count_words()
        seo_score = self.calculate_seo_score()
        readability_score = self.calculate_readability_score()

        # Análise de densidade de palavras-chave
        keyword_density = {}
        text_lower = self.article.lower()
        for keyword in self.metadata["keywords"]:
            keyword_lower = keyword.lower()
            count = text_lower.count(keyword_lower)
            density = count / self.metadata["word_count"] * 100
            keyword_density[keyword] = {
                "count": count,
                "density": round(density, 2)
            }

        # Prepara sugestões de melhoria
        suggestions = []
        if seo_score < 70:
            if len(re.findall(r'##\s+', self.article)) < 4:
                suggestions.append("Adicione mais seções com subtítulos H2 para melhorar a estrutura do artigo.")

            low_density_keywords = [k for k, v in keyword_density.items() if v["density"] < 0.5]
            if low_density_keywords:
                suggestions.append(f"Aumente a frequência das seguintes palavras-chave: {', '.join(low_density_keywords)}")

            high_density_keywords = [k for k, v in keyword_density.items() if v["density"] > 2.0]
            if high_density_keywords:
                suggestions.append(f"Reduza a frequência das seguintes palavras-chave (possível keyword stuffing): {', '.join(high_density_keywords)}")

        if readability_score < 70:
            if self.avg_word_length > 6:
                suggestions.append("Use palavras mais simples e curtas para melhorar a legibilidade.")
            if self.words_per_sentence > 20:
                suggestions.append("Reduza o comprimento das frases para melhorar a legibilidade.")

        return {
            "title": self.metadata["title"],
            "word_count": self.metadata["word_count"],
            "seo_score": self.metadata["seo_score"],
            "readability_score": self.metadata["readability_score"],
            "keyword_density": keyword_density,
            "improvement_suggestions": suggestions
        }

    def save_article(self, format="markdown"):
        """
        Salva o artigo no formato especificado.

        Args:
            format (str): Formato de saída ('markdown', 'html', 'text')

        Returns:
            str: Caminho do arquivo salvo.
        """
        if not self.article:
            return {"error": "Nenhum artigo gerado ainda."}

        filename = re.sub(r'[^\w\s]', '', self.metadata["title"])
        filename = re.sub(r'\s+', '_', filename.lower())

        if format == "markdown":
            filename = f"{filename}.md"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(self.article)

        elif format == "html":
            filename = f"{filename}.html"

            # Conversão básica de Markdown para HTML
            html = self.article
            html = re.sub(r'# (.*?)\n', r'<h1>\1</h1>\n', html)
            html = re.sub(r'## (.*?)\n', r'<h2>\1</h2>\n', html)
            html = re.sub(r'### (.*?)\n', r'<h3>\1</h3>\n', html)
            html = re.sub(r'\n\n(.*?)\n\n', r'<p>\1</p>\n', html)

            with open(filename, "w", encoding="utf-8") as f:
                f.write(f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{self.metadata["title"]}</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #333; }}
        h2 {{ color: #444; margin-top: 30px; }}
        h3 {{ color: #555; }}
        p {{ margin-bottom: 20px; }}
    </style>
</head>
<body>
{html}
</body>
</html>""")

        elif format == "text":
            filename = f"{filename}.txt"
            # Remove marcadores de Markdown
            text = re.sub(r'#+ ', '', self.article)
            with open(filename, "w", encoding="utf-8") as f:
                f.write(text)

        # Para uso no Google Colab, permitir download do arquivo
        if IN_COLAB:
            try:
                from google.colab import files
                files.download(filename)
                return {"message": f"Arquivo '{filename}' criado e disponível para download."}
            except Exception as e:
                return {"message": f"Arquivo '{filename}' criado, mas ocorreu um erro ao tentar o download: {str(e)}"}
        else:
            return {"message": f"Arquivo '{filename}' criado com sucesso."}

# Adiciona código para configurar as chaves API no Colab (adicione no início do notebook)
def setup_api_keys():
    """Configura as chaves API da OpenAI e Serper no Google Colab usando userdata."""
    if not IN_COLAB:
        print("Esta função só funciona no ambiente Google Colab.")
        return

    print("\n===== 🔑 CONFIGURAÇÃO DAS CHAVES API =====\n")

    # Configuração da chave OpenAI
    print("=== Configuração da chave OpenAI ===")
    print("Esta função irá configurar sua chave API da OpenAI de forma segura no Google Colab.")
    print("Sua chave será armazenada nas variáveis secretas (Secrets) e não ficará visível no notebook.")

    # Verifica se já existe uma chave configurada
    try:
        existing_key = userdata.get('OPENAI_API_KEY')
        if existing_key:
            print("✅ Já existe uma chave API OpenAI configurada.")
            replace = input("Deseja substituir a chave existente? (s/n): ").lower()
            if replace != 's':
                print("Mantendo a chave existente.")
            else:
                setup_openai_key()
        else:
            setup_openai_key()
    except:
        setup_openai_key()

    # Configuração da chave Serper
    print("\n=== Configuração da chave Serper para busca na web ===")
    print("Esta função irá configurar sua chave API Serper de forma segura no Google Colab.")
    print("Sua chave será armazenada nas variáveis secretas (Secrets) e não ficará visível no notebook.")

    use_serper = input("Deseja configurar uma chave API Serper para busca na web? (s/n): ").lower()
    if use_serper == 's':
        # Verifica se já existe uma chave configurada
        try:
            existing_key = userdata.get('SERPER_API_KEY')
            if existing_key:
                print("✅ Já existe uma chave API Serper configurada.")
                replace = input("Deseja substituir a chave existente? (s/n): ").lower()
                if replace != 's':
                    print("Mantendo a chave existente.")
                else:
                    setup_serper_key()
            else:
                setup_serper_key()
        except:
            setup_serper_key()

    print("\nConfigurações concluídas.")
    print("Agora você pode executar o gerador de artigos de blog com as chaves armazenadas de forma segura.")

def setup_openai_key():
    """Configura a chave API da OpenAI."""
    new_key = input("\nDigite sua chave API da OpenAI (começando com 'sk-'): ")

    if not new_key.startswith('sk-'):
        print("⚠️ Aviso: A chave não começa com 'sk-', o que é o padrão para chaves OpenAI.")
        confirm = input("Tem certeza de que esta é uma chave válida? (s/n): ").lower()
        if confirm != 's':
            print("Operação cancelada.")
            return

    print("\nPara salvar a chave no Google Colab:")
    print("1. No menu lateral, clique em '🔑' (Secrets)")
    print("2. Adicione uma nova chave secreta com:")
    print("   - Nome: OPENAI_API_KEY")
    print(f"   - Valor: {new_key}")
    print("3. Clique em 'Adicionar'")

    # Testa a chave (opcional)
    test = input("\nDeseja testar a chave agora? (s/n): ").lower()
    if test == 's':
        try:
            client = OpenAI(api_key=new_key)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "Responda apenas com 'OK' se você pode me ver."}],
                max_tokens=10
            )
            result = response.choices[0].message.content
            if "ok" in result.lower():
                print("✅ Teste bem-sucedido! A chave API está funcionando.")
            else:
                print(f"⚠️ A API respondeu com: {result}")
        except Exception as e:
            print(f"❌ Erro ao testar a chave: {str(e)}")

def setup_serper_key():
    """Configura a chave API da Serper."""
    new_key = input("\nDigite sua chave API da Serper: ")

    print("\nPara salvar a chave no Google Colab:")
    print("1. No menu lateral, clique em '🔑' (Secrets)")
    print("2. Adicione uma nova chave secreta com:")
    print("   - Nome: SERPER_API_KEY")
    print(f"   - Valor: {new_key}")
    print("3. Clique em 'Adicionar'")

    # Testa a chave (opcional)
    test = input("\nDeseja testar a chave agora? (s/n): ").lower()
    if test == 's':
        try:
            headers = {
                "X-API-KEY": new_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": "teste serper api",
                "num": 1
            }
            response = requests.post(
                "https://google.serper.dev/search",
                headers=headers,
                json=payload
            )

            if response.status_code == 200:
                print("✅ Teste bem-sucedido! A chave API Serper está funcionando.")
            else:
                print(f"❌ Erro: {response.status_code}")
                print(response.text)
        except Exception as e:
            print(f"❌ Erro ao testar a chave: {str(e)}")

# Interface principal para uso no Google Colab
def run_blog_article_generator():
    """Interface principal para o gerador de artigos de blog."""
    print("\n===== 📝 GERADOR DE ARTIGOS DE BLOG OTIMIZADOS PARA SEO =====\n")

    # Obtém a chave API da OpenAI
    openai_api_key = None
    if IN_COLAB:
        try:
            openai_api_key = userdata.get('OPENAI_API_KEY')
            print("✅ Chave API OpenAI obtida com sucesso do armazenamento seguro do Colab.")
        except Exception as e:
            print(f"⚠️ Não foi possível obter a chave API do armazenamento seguro: {str(e)}")
            print("\nPara configurar uma chave secreta no Google Colab:")
            print("1. No menu lateral, clique em '🔑' (Secrets)")
            print("2. Adicione uma nova chave secreta com nome 'OPENAI_API_KEY' e seu valor")
            openai_api_key = input("\nDigite sua chave de API OpenAI: ")
    else:
        openai_api_key = input("Digite sua chave de API OpenAI: ")

    # Verifica se deseja usar a busca na internet
    use_internet_search = input("\nDeseja usar a busca na internet para enriquecer o conteúdo? (s/n): ").lower() == 's'

    serper_api_key = None
    if use_internet_search:
        if IN_COLAB:
            try:
                serper_api_key = userdata.get('SERPER_API_KEY')
                print("✅ Chave API Serper obtida com sucesso do armazenamento seguro do Colab.")
            except Exception as e:
                print(f"⚠️ Não foi possível obter a chave API Serper do armazenamento seguro: {str(e)}")
                print("\nPara configurar uma chave secreta no Google Colab:")
                print("1. No menu lateral, clique em '🔑' (Secrets)")
                print("2. Adicione uma nova chave secreta com nome 'SERPER_API_KEY' e seu valor")
                serper_api_key = input("\nDigite sua chave de API Serper: ")
        else:
            serper_api_key = input("Digite sua chave de API Serper: ")

    # Seleciona o modelo
    print("\nSelecione o modelo da OpenAI:")
    print("\nReasoning models:")
    print("1. o3-mini - Fast, flexible, intelligent reasoning model")
    print("2. o1 - High-intelligence reasoning model")
    print("3. o1-mini - A faster, more affordable reasoning model than o1")
    print("4. o1-pro - A version of o1 with more compute for better responses")

    print("\nFlagship chat models:")
    print("5. GPT-4.5 Preview - Largest and most capable GPT model")
    print("6. GPT-4o - Fast, intelligent, flexible GPT model")
    print("7. ChatGPT-4o - GPT-4o model used in ChatGPT")

    print("\nCost-optimized models:")
    print("8. GPT-4o mini - Fast, affordable small model for focused tasks")

    print("\nOlder GPT models:")
    print("9. GPT-4 Turbo - An older high-intelligence GPT model")
    print("10. GPT-4 - An older high-intelligence GPT model")
    print("11. GPT-3.5 Turbo - Legacy GPT model for cheaper chat and non-chat tasks")

    model_choice = input("\nEscolha o número correspondente (padrão: 11): ") or "11"

    models = {
        "1": "o3-mini",
        "2": "o1",
        "3": "o1-mini",
        "4": "o1-pro",
        "5": "gpt-4.5-preview",
        "6": "gpt-4o",
        "7": "gpt-4o",  # ChatGPT-4o usa o mesmo modelo que GPT-4o
        "8": "gpt-4o-mini",
        "9": "gpt-4-turbo",
        "10": "gpt-4",
        "11": "gpt-3.5-turbo"
    }

    selected_model = models.get(model_choice, "gpt-3.5-turbo")
    print(f"Modelo selecionado: {selected_model}")

    # Inicializa o gerador
    generator = BlogArticleGenerator(
        openai_api_key=openai_api_key,
        serper_api_key=serper_api_key,
        model=selected_model
    )

    if use_internet_search:
        generator.enable_internet_search(serper_api_key)

    # Solicita informações básicas
    topic = input("\nQual é o tópico principal do artigo? ")

    print("\nInsira as palavras-chave para SEO (separadas por vírgula): ")
    keywords_input = input()
    keywords = [k.strip() for k in keywords_input.split(",")]

    tone_options = ["informativo", "persuasivo", "conversacional", "técnico", "inspirador"]
    print("\nSelecione o tom do artigo:")
    for i, tone in enumerate(tone_options):
        print(f"{i+1}. {tone}")

    tone_idx = int(input("Escolha o número correspondente: ")) - 1
    tone = tone_options[tone_idx]

    # Cria o esboço
    print("\nCriando esboço do artigo...")

    # Tenta até 3 vezes com diferentes modelos se falhar
    attempts = 0
    outline = None
    available_models = ["gpt-3.5-turbo", "gpt-4o-mini", "gpt-4o"]

    while attempts < 3 and outline is None:
        try:
            outline = generator.create_outline(topic, keywords, tone, use_search=use_internet_search)
            if outline:
                break
        except Exception as e:
            print(f"Tentativa {attempts+1} falhou: {str(e)}")

        attempts += 1
        if attempts < 3:
            # Tenta com um modelo diferente
            fallback_model = available_models[attempts % len(available_models)]
            print(f"Tentando novamente com o modelo {fallback_model}...")
            generator.model = fallback_model

    if not outline:
        print("❌ Erro ao criar o esboço após várias tentativas. Verifique sua chave de API e tente novamente.")
        return

    # Mostra o esboço
    print("\n===== ESBOÇO DO ARTIGO =====")
    print(f"📌 Título: {outline['title']}")
    print(f"📝 Meta Descrição: {outline.get('meta_description', 'Não disponível')}")
    print("\n📑 Seções:")
    for i, section in enumerate(outline['sections']):
        print(f"  {i+1}. {section['heading']}")
        for point in section['points']:
            print(f"     • {point}")

    # Pergunta se deseja prosseguir
    proceed = input("\nDeseja gerar o artigo completo com base neste esboço? (s/n): ")
    if proceed.lower() != 's':
        print("Operação cancelada.")
        return

    # Gera o artigo
    article = generator.generate_full_article(outline, use_search=use_internet_search)

    # Mostra estatísticas
    analysis = generator.analyze_article()

    print("\n===== ANÁLISE DO ARTIGO =====")
    print(f"📊 Contagem de palavras: {analysis['word_count']} (mínimo: {generator.min_word_count})")
    print(f"🔍 Pontuação SEO: {analysis['seo_score']}/100")
    print(f"📖 Pontuação de legibilidade: {analysis['readability_score']}/100")

    print("\n📈 Densidade de palavras-chave:")
    for keyword, data in analysis['keyword_density'].items():
        status = "✅" if 0.5 <= data['density'] <= 2.0 else "⚠️"
        print(f"  {status} '{keyword}': {data['count']} ocorrências ({data['density']}%)")

    if analysis['improvement_suggestions']:
        print("\n🛠️ Sugestões de melhoria:")
        for suggestion in analysis['improvement_suggestions']:
            print(f"  • {suggestion}")

    # Exibe uma prévia do artigo
    print("\n===== PRÉVIA DO ARTIGO =====")
    preview_lines = article.split("\n")[:20]
    preview = "\n".join(preview_lines)
    print(f"{preview}\n...\n")

    # Opções de salvar
    print("Deseja salvar o artigo? Escolha o formato:")
    print("1. Markdown (.md)")
    print("2. HTML (.html)")
    print("3. Texto (.txt)")
    print("4. Mostrar artigo completo na tela")
    print("5. Não salvar")

    save_option = int(input("Escolha o número correspondente: "))

    if save_option == 1:
        result = generator.save_article("markdown")
        print(result["message"])
    elif save_option == 2:
        result = generator.save_article("html")
        print(result["message"])
    elif save_option == 3:
        result = generator.save_article("text")
        print(result["message"])
    elif save_option == 4:
        display(Markdown(article))
    else:
        print("Artigo não salvo.")

    print("\n✅ Operação concluída!")

# Para uso direto com o gerador de artigos
if __name__ == "__main__":
    if IN_COLAB:
        print("Detectado ambiente Google Colab.")
        print("Para configurar suas chaves API de forma segura, execute a função setup_api_keys() primeiro.")

    run_blog_article_generator()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Detectado ambiente Google Colab.
Para configurar suas chaves API de forma segura, execute a função setup_api_keys() primeiro.

===== 📝 GERADOR DE ARTIGOS DE BLOG OTIMIZADOS PARA SEO =====

✅ Chave API OpenAI obtida com sucesso do armazenamento seguro do Colab.

Deseja usar a busca na internet para enriquecer o conteúdo? (s/n): s
✅ Chave API Serper obtida com sucesso do armazenamento seguro do Colab.

Selecione o modelo da OpenAI:

Reasoning models:
1. o3-mini - Fast, flexible, intelligent reasoning model
2. o1 - High-intelligence reasoning model
3. o1-mini - A faster, more affordable reasoning model than o1
4. o1-pro - A version of o1 with more compute for better responses

Flagship chat models:
5. GPT-4.5 Preview - Largest and most capable GPT model
6. GPT-4o - Fast, intelligent, flexible GPT model
7. ChatGPT-4o - GPT-4o model used in ChatGPT

Cost-optimized models:
8. GPT-4o mini - Fast, affordable small model for focused tasks

Older GPT models:
9. GPT-4 Turbo - An older high-intell

100%|██████████| 9/9 [08:22<00:00, 55.82s/it]



===== ANÁLISE DO ARTIGO =====
📊 Contagem de palavras: 4786 (mínimo: 2000)
🔍 Pontuação SEO: 79.67/100
📖 Pontuação de legibilidade: 77.53/100

📈 Densidade de palavras-chave:
  ✅ 'openai': 27 ocorrências (0.56%)
  ✅ 'meta': 28 ocorrências (0.59%)
  ✅ 'google': 24 ocorrências (0.5%)
  ✅ 'hugging face': 34 ocorrências (0.71%)
  ⚠️ 'elon musk': 15 ocorrências (0.31%)

===== PRÉVIA DO ARTIGO =====
# Open Source LLM: O Futuro Aberto da Inteligência Artificial

## Introdução

## Introdução

Nos últimos anos, a inteligência artificial (IA) deu um salto impressionante graças ao surgimento dos Modelos de Linguagem de Grande Porte, conhecidos como LLM (do inglês, Large Language Models). Esses modelos são sistemas de IA treinados em imensas quantidades de texto para gerar respostas coerentes, realistas e contextualmente relevantes. Exemplos famosos incluem o GPT-4 da OpenAI, LaMDA do Google e LLaMA da Meta, que têm transformado significativamente a forma como interagimos com tecnologia no dia a dia

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo 'open_source_llm_o_futuro_aberto_da_inteligência_artificial.html' criado e disponível para download.

✅ Operação concluída!
